In [1]:
import pandas as pd
import numpy as np
import requests
import codecs
import time
import bs4
import re
import json
from os import listdir
from os.path import isfile, join
import pymystem3
import warnings

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression,SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier

In [2]:
# словари
negation = {"не"}
stopwords = {'c',  'а',  'але',  'без',  'белый',  'близко',  'более',  'больше',  'большой',  'будто',  'бы',  'бывать',  'бывь',  'быть',  'в',  'важный',  'ваш',  'вверх',  'вдали',  'вдруг',  'ведь',  'везде',  'вернуться',  'весь',  'вечер',  'взгляд',  'взять',  'вид',  'видеть',  'вместе',  'вне',  'вниз',  'внизу',  'во',  'вода',  'война',  'вокруг',  'вон',  'вообще',  'вопрос',  'восемнадцатый',  'восемнадцать',  'восемь',  'восьмой',  'вот',  'впрочем',  'время',  'все',  'всегда',  'всего',  'всюду',  'второй',  'вы',  'выходить',  'г',  'где',  'главный',  'глаз',  'говорить',  'год',  'голова',  'голос',  'город',  'да',  'давать',  'давно',  'даже',  'далекий',  'далеко',  'даром',  'два',  'двадцатый',  'двадцать',  'двенадцатый',  'двенадцать',  'дверь',  'девятнадцатый',  'девятнадцать',  'девятый',  'девять',  'действительно',  'делать',  'дело',  'день',  'деньги',  'десятый',  'десять',  'для',  'до',  'довольно',  'долго',  'должно',  'должный',  'дом',  'дорога',  'друг',  'друго',  'другой',  'думать',  'душа',  'е',  'если',  'еще',  'ж',  'ждать',  'же',  'жена',  'женщина',  'жизнь',  'жить',  'за',  'занимать',  'занятый',  'затем',  'зато',  'зачем',  'здесь',  'земля',  'знать',  'значит',  'значить',  'и',  'идти',  'из',  'или',  'именно',  'иметь',  'имя',  'иногда',  'к',  'каждый',  'кажется',  'казаться',  'как',  'какой',  'книга',  'когда',  'ком',  'комната',  'конец',  'конечно',  'который',  'кроме',  'кругом',  'кто',  'куда',  'лежать',  'ли',  'лицо',  'лишь',  'любить',  'м',  'маленький',  'мало',  'мать',  'машина',  'между',  'менее',  'место',  'миллион',  'мимо',  'минута',  'мир',  'много',  'многочисленный',  'мож',  'может',  'можно',  'можхо',  'мой',  'молоть',  'мор',  'москва',  'мочь',  'мы',  'на',  'наверху',  'над',  'надо',  'назад',  'наиболее',  'наконец',  'народ',  'находить',  'начинать',  'наш',  'недавно',  'недалеко',  'некоторый',  'нельзя',  'немного',  'немой',  'непрерывный',  'нередко',  'несколько',  'нет',  'ни',  'нибудь',  'ниже',  'низко',  'никакой',  'никогда',  'никто',  'никуда',  'ничего',  'ничто',  'но',  'новый',  'нога',  'ночь',  'ну',  'нужно',  'нужный',  'нх',  'о',  'об',  'оба',  'обычно',  'один',  'одиннадцатый',  'одиннадцать',  'однажды',  'однако',  'оказываться',  'окно',  'около',  'он',  'она',  'они',  'оно',  'опять',  'особенно',  'оставаться',  'от',  'отвечать',  'отец',  'откуда',  'отовсюду',  'отсюда',  'очень',  'первый',  'перед',  'писать',  'плечо',  'по',  'под',  'подумать',  'подходить',  'пожалуйста',  'поздно',  'пойти',  'пока',  'пол',  'получать',  'помнить',  'понимать',  'пора',  'после',  'последний',  'посмотреть',  'посреди',  'потом',  'потому',  'почему',  'почти',  'правда',  'прекрасно',  'при',  'про',  'просто',  'против',  'процент',  'путь',  'пятнадцатый',  'пятнадцать',  'пятый',  'пять',  'работа',  'работать',  'раз',  'разве',  'рано',  'ребенок',  'решать',  'россия',  'рука',  'русский',  'ряд',  'рядом',  'с',  'сам',  'самый',  'свет',  'свое',  'свой',  'сделать',  'сеаой',  'себя',  'сегодня',  'седьмой',  'сей',  'сейчас',  'семнадцатый',  'семнадцать',  'семь',  'сидеть',  'сила',  'сказать',  'сколько',  'слишком',  'слово',  'случай',  'смотреть',  'сначала',  'снова',  'со',  'советский',  'совсем',  'спасибо',  'спрашивать',  'сразу',  'становиться',  'старый',  'стол',  'сторона',  'стоять',  'страна',  'суть',  'считать',  'т',  'так',  'также',  'таки',  'такой',  'там',  'твой',  'теперь',  'то',  'товарищ',  'тогда',  'тоже',  'только',  'том',  'тот',  'третий',  'три',  'тринадцатый',  'тринадцать',  'туда',  'тут',  'ты',  'тысяча',  'у',  'увидеть',  'уж',  'уже',  'улица',  'уметь',  'утро',  'хороший',  'хорошо',  'хотеть',  'хоть',  'хотя',  'час',  'часто',  'часть',  'человек',  'через',  'четвертый',  'четыре',  'четырнадцатый',  'четырнадцать',  'что',  'чтоб',  'чтобы',  'чуть',  'шестнадцатый',  'шестнадцать',  'шестой',  'шесть',  'это',  'этот',  'я',  'являться'}

token_list = []
rgx_tokens = re.compile(pattern="\w+")
rgx_number = re.compile("[\d\W\-]+")

def preprocesing(text):
    tokens = rgx_tokens.findall(text)
    is_negation = False
    result = []
    for token in tokens: 
        morph_info = morph.analyze(token)
        if morph_info and morph_info[0].get("analysis", False)\
            and morph_info[0]["analysis"] \
            and morph_info[0]["analysis"][0].get("lex", False):
            lemma = morph_info[0]["analysis"][0]["lex"]
        else:
            continue
        if lemma in stopwords:
            is_negation = False
            continue
        if token.lower() in negation:
            is_negation = True
            continue
        elif is_negation:
            is_negation = False
            lemma = "NEG_"+lemma
        
        result.append(lemma)
    return " ".join(result)


In [3]:
# загрузка тестовых данных
with open('test.csv') as f:
    text = f.read()
    parser = bs4.BeautifulSoup(text,'lxml')
    txt_test = list(x.get_text() for x in  parser.find_all('review'))
    del text
    del parser

In [4]:
#загрузка тренировочных данных

with open("./reviews01.json") as f:
    txt_train1 = json.load(f)

with open("./citilink_reviews.json") as f:
    txt_train2 = json.load(f)

#разбивка на положительный и отрицательный классы
pos_docs = []
neg_docs = []
for txt_train in [txt_train1, txt_train2]:
    for rec in txt_train:
        pos_docs.append(rec["advantages"])
        neg_docs.append(rec["disadvantages"])
        if int(rec["rating"])>3:
            pos_docs.append(rec["comment"])
        elif int(rec["rating"])<3:
            neg_docs.append(rec["comment"])

dtrain = pd.concat(
    objs=[
        pd.DataFrame({
            "text":pos_docs
        }).assign(response=1),
        pd.DataFrame({
            "text":neg_docs
        }).assign(response=0)
    ]
)

dtest = pd.DataFrame({
    "Id": list(range(len(txt_test))),
    "text": txt_test
})

morph = pymystem3.Mystem()
# dtrain["text"] = dtrain.text.apply(preprocesing)
# dtest["text"] = dtest.text.apply(preprocesing)

dtrain = dtrain[dtrain.text.notnull()]

In [99]:
# warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.simplefilter(action='ignore', category=warnings.ConvergenceWarning)
token_pattern = r'(?u)\b\w\w+\b|!|\?|\)|\%'
models = [
    ('LogisticRegression', LogisticRegression()),
#     ('LogisticRegressionCV', LogisticRegressionCV()),
    ('SGDClassifier', SGDClassifier()),
    ('LinearSVC', LinearSVC()),
    ("BernoulliNB", BernoulliNB()),
]

for name, model in models:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        count_vec_uni = CountVectorizer(token_pattern=token_pattern) 
        tfidf_vec_uni = TfidfVectorizer(token_pattern=token_pattern)
        count_vec_bi = CountVectorizer(ngram_range=(1, 2)) 
        tfidf_vec_bi = TfidfVectorizer(ngram_range=(1, 2))

        x_count_uni = count_vec_uni.fit_transform(dtrain['text'])
        x_tfidf_uni = tfidf_vec_uni.fit_transform(dtrain['text'])
        x_count_bi = count_vec_bi.fit_transform(dtrain['text'])
        x_tfidf_bi = tfidf_vec_bi.fit_transform(dtrain['text'])

        res_count_uni = cross_val_score(model,x_count_uni,dtrain['response'],cv=5,scoring='accuracy').mean()
        res_tfidf_uni = cross_val_score(model,x_tfidf_uni,dtrain['response'],cv=5,scoring='accuracy').mean()
        res_count_bi  = cross_val_score(model,x_count_bi,dtrain['response'],cv=5,scoring='accuracy').mean()
        res_tfidf_bi  = cross_val_score(model,x_tfidf_bi,dtrain['response'],cv=5,scoring='accuracy').mean()

        print(f'(count,{name},uni)={res_count_uni}')
        print(f'(tfidf,{name},uni)={res_tfidf_uni}')
        print(f'(count,{name},bi) ={res_count_bi}')
        print(f'(tfidf,{name},bi) ={res_tfidf_bi}')
        print("")

(count,LogisticRegression,uni)=0.8551246713212597
(tfidf,LogisticRegression,uni)=0.8502455582614669
(count,LogisticRegression,bi) =0.8596577828052405
(tfidf,LogisticRegression,bi) =0.8092763769114171

(count,SGDClassifier,uni)=0.8345509208131293
(tfidf,SGDClassifier,uni)=0.8490243776793098
(count,SGDClassifier,bi) =0.8469311979400089
(tfidf,SGDClassifier,bi) =0.8697694535638029

(count,LinearSVC,uni)=0.8340326820002304
(tfidf,LinearSVC,uni)=0.8539048569397348
(count,LinearSVC,bi) =0.8427513831827127
(tfidf,LinearSVC,bi) =0.8688983728942784

(count,BernoulliNB,uni)=0.8397802843486046
(tfidf,BernoulliNB,uni)=0.8397802843486046
(count,BernoulliNB,bi) =0.679037923794495
(tfidf,BernoulliNB,bi) =0.679037923794495



# Проверка SGDClassifier

In [111]:
SGDClassifier()

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [79]:
for c in [.01, .05, .1, .25, .5, .75, .8, .85, .9, .95, 1, 1.25, 5, 10]:
    tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
    X_tfidf = tfidf_vec.fit_transform(dtrain['text'])
    model = SGDClassifier(C=c, )
    
    print(f"Arruracy[C={c}]={cross_val_score(model,X_tfidf,dtrain['response'],cv=5,scoring='accuracy').mean()}")

Arruracy[C=0.01]=0.6327217591393262
Arruracy[C=0.05]=0.732471846761044
Arruracy[C=0.1]=0.8207433804785342
Arruracy[C=0.25]=0.8424459194402658
Arruracy[C=0.5]=0.8464096864992893
Arruracy[C=0.75]=0.847868234351632
Arruracy[C=0.8]=0.8482859885797881
Arruracy[C=0.85]=0.8487033074205256
Arruracy[C=0.9]=0.8482859885797879
Arruracy[C=0.95]=0.8484951922344299
Arruracy[C=1]=0.8474517862857317
Arruracy[C=1.25]=0.8472428003247992
Arruracy[C=5]=0.8409810584703534
Arruracy[C=10]=0.8401462030951692


In [119]:
parameters = {
    'max_iter': (None,),
    'alpha': (1e-6, 1e-5, 1e-3,),
    'penalty': ('l2', 'elasticnet'),
    'epsilon': (0.5, 0.25, 0.1, 0.01, 1e-4 )
}
tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
X_tfidf = tfidf_vec.fit_transform(dtrain['text'])
grid_search = GridSearchCV(SGDClassifier(), parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_tfidf, dtrain.response.tolist())

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    2.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_iter': (None,), 'alpha': (1e-06, 1e-05, 0.001), 'penalty': ('l2', 'elasticnet'), 'epsilon': (0.5, 0.25, 0.1, 0.01, 0.0001)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [120]:
grid_search.best_params_

{'alpha': 1e-05, 'epsilon': 0.01, 'max_iter': None, 'penalty': 'elasticnet'}

In [121]:
best_params = {'alpha': 1e-05, 'epsilon': 0.01, 'max_iter': None, 'penalty': 'elasticnet'}
for n in list(range(1,100,10)):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
    X_tfidf = tfidf_vec.fit_transform(dtrain['text'])
    model = BaggingClassifier(base_estimator=SGDClassifier(**best_params),n_estimators=n,bootstrap_features=True,bootstrap=True)
    
    print(f"Arruracy[n={n}]={cross_val_score(model,X_tfidf,dtrain['response'],cv=5,scoring='accuracy').mean()}")

Arruracy[n=1]=0.8021313957162384
Arruracy[n=11]=0.8523376702754775
Arruracy[n=21]=0.8594859991128336
Arruracy[n=31]=0.8607085496066841
Arruracy[n=41]=0.8622760371319653
Arruracy[n=51]=0.8638464129230767
Arruracy[n=61]=0.8619295809511701
Arruracy[n=71]=0.8668100610068225
Arruracy[n=81]=0.8641937812296725
Arruracy[n=91]=0.865415421188177


In [13]:
best_params = {'alpha': 1e-05, 'epsilon': 0.01, 'max_iter': None, 'penalty': 'elasticnet'}
n = 200
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))
X_tfidf = tfidf_vec.fit_transform(dtrain['text'])
model = BaggingClassifier(
    base_estimator=SGDClassifier(**best_params),
    n_estimators=n,bootstrap_features=True,bootstrap=True
)

print(f"Arruracy[n={n}]={cross_val_score(model,X_tfidf,dtrain['response'],cv=5,scoring='accuracy').mean()}")

Arruracy[n=200]=0.8549566861649527


In [14]:
# model = BaggingClassifier(
#     base_estimator=SGDClassifier(**best_params),
#     n_estimators=n,bootstrap_features=True,bootstrap=True
# )

# tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
# train_matrix = tfidf_vec.fit_transform(dtrain['text'])
# test_matrix = tfidf_vec.transform(dtest['text'])

# model.fit(train_matrix, dtrain.response)
# dtest["pred"]=model.predict(test_matrix)
# dtest["y"] = dtest.pred.apply(lambda x: "pos" if x==1 else "neg")
# dtest[["Id", "y"]].to_csv("submition03.csv", index=None)

# Проверка LinearSVC

In [16]:
best_params = {'alpha': 1e-05, 'epsilon': 0.01, 'max_iter': None, 'penalty': 'elasticnet'}
n = 200
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))
X_tfidf = tfidf_vec.fit_transform(dtrain['text'])
model = LinearSVC()

print(f"Arruracy[n={n}]={cross_val_score(model,X_tfidf,dtrain['response'],cv=5,scoring='accuracy').mean()}")

Arruracy[n=200]=0.8654113180799922


In [17]:
model = LinearSVC()

tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
train_matrix = tfidf_vec.fit_transform(dtrain['text'])
test_matrix = tfidf_vec.transform(dtest['text'])

model.fit(train_matrix, dtrain.response)
dtest["pred"]=model.predict(test_matrix)
dtest["y"] = dtest.pred.apply(lambda x: "pos" if x==1 else "neg")


dtest[["Id", "y"]].to_csv("submition04.csv", index=None)

In [18]:
dtest

,Id,text,pred,y
0,0,"Ужасно слабый аккумулятор, это основной минус ...",0,neg
1,1,ценанадежность-неубиваемостьдолго держит батар...,1,pos
2,2,"подробнее в комментариях\nК сожалению, факт по...",0,neg
3,3,я любительница громкой музыки. Тише телефона у...,0,neg
4,4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1,pos
5,5,- Удобная Клавиатура и русская раскладка\n- 2 ...,1,pos
6,6,Супер телефон!\n1.QWERTY!!! Это самый лучший е...,1,pos
7,7,- толщина (помещается даже в брюки)\n- аккумул...,1,pos
8,8,Аккумулятор ужасен! Хватает буквально на неско...,0,neg
9,9,1 удобный.клавеатура просто класс быстро пишеш...,1,pos
